<a href="https://colab.research.google.com/github/alivarastepour/diabetes_prediction/blob/master/diabetes_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Purpose of this notebook
This notebook aims to build a model that determines whether a person is prone to diabetes or not. Additionally, it seeks to identify a subset of features (risk factors) that can accurately predict the risk of diabetes. The weights of the optimal solution will be utilized in another project, where they will be applied to users' inputs in real time.

## Dataset
This notebook makes use of a subset of a larger dataset which aimed to collect uniform, state-specific data on preventive health practices and risk behaviors that are associated with chronic diseases, injuries, and preventable infectious diseases in the adult population. The subset used in this notebook can be accessed [here](https://www.kaggle.com/datasets/alexteboul/diabetes-health-indicators-dataset?select=diabetes_binary_5050split_health_indicators_BRFSS2015.csv).

In [1]:
import pandas as pd
import numpy as np
from google.colab import drive

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense,LeakyReLU,Dropout
from keras.optimizers import Adagrad, RMSprop, Adam
from keras.regularizers import l1, l2
from keras.initializers import he_normal
from keras.activations import selu
from keras.metrics import Precision, Recall

In [2]:
drive.mount('/drive')
DATASET_ADDRESS = '/drive/MyDrive/diabetes_info.csv'
raw_dataset = pd.read_csv(DATASET_ADDRESS)

Mounted at /drive


In [3]:
raw_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70692 entries, 0 to 70691
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Diabetes_binary       70692 non-null  float64
 1   HighBP                70692 non-null  float64
 2   HighChol              70692 non-null  float64
 3   CholCheck             70692 non-null  float64
 4   BMI                   70692 non-null  float64
 5   Smoker                70692 non-null  float64
 6   Stroke                70692 non-null  float64
 7   HeartDiseaseorAttack  70692 non-null  float64
 8   PhysActivity          70692 non-null  float64
 9   Fruits                70692 non-null  float64
 10  Veggies               70692 non-null  float64
 11  HvyAlcoholConsump     70692 non-null  float64
 12  AnyHealthcare         70692 non-null  float64
 13  NoDocbcCost           70692 non-null  float64
 14  GenHlth               70692 non-null  float64
 15  MentHlth           

In [4]:
y = raw_dataset["Diabetes_binary"]
x = raw_dataset.drop(columns=["Diabetes_binary"])

In [5]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=9)

# Model selection
While our data may appear relatively clean, this does not guarantee optimal performance. Therefore, we must leverage a range of machine learning models to assess their effectiveness and identify potential modifications to the original data that can enhance the performance of our models.

## First model: Gradient boost classifier
Boosting algorithms have been widely recognized as effective choices for handling tabular data. Among them, gradient boosting stands out as a prominent technique that leverages decision trees to create a powerful ensemble model. Nonetheless, to ensure its optimal performance, careful consideration should be given to hyperparameter tuning.

In [6]:
def get_data(dataset):
  y = dataset["Diabetes_binary"]
  x = dataset.drop(columns=["Diabetes_binary"])
  return train_test_split(x, y, test_size=0.20, random_state=9)

In [73]:
def gradienBoostClassifierModel(dataset, learning_rate=0.05, n_estimators=150, subsample=0.8):
  x_train, x_test, y_train, y_test = get_data(dataset)
  reg = GradientBoostingClassifier(random_state=90,
                                loss='deviance',
                                learning_rate=learning_rate,
                                n_estimators=n_estimators,
                                subsample=subsample,
                                criterion='friedman_mse',
                                verbose=2,
                                )
  reg.fit(x_train, y_train)
  y_pred = reg.predict(x_test)
  report = classification_report(y_test, y_pred)
  print(report)

In [8]:
gradienBoostClassifierModel(raw_dataset)

      Iter       Train Loss      OOB Improve   Remaining Time 
         1           1.3619           0.0245            7.23s
         2           1.3400           0.0224            7.08s


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py:280: FutureWarning: The loss parameter name 'deviance' was deprecated in v1.1 and will be removed in version 1.3. Use the new parameter name 'log_loss' which is equivalent.
  warnings.warn(


         3           1.3200           0.0203            7.03s
         4           1.3008           0.0178            6.92s
         5           1.2841           0.0163            6.93s
         6           1.2685           0.0151            6.84s
         7           1.2545           0.0136            6.75s
         8           1.2417           0.0127            6.74s
         9           1.2303           0.0117            6.69s
        10           1.2191           0.0112            6.73s
        11           1.2090           0.0101            6.66s
        12           1.1982           0.0100            6.64s
        13           1.1890           0.0094            6.59s
        14           1.1793           0.0088            6.57s
        15           1.1707           0.0078            6.54s
        16           1.1641           0.0067            6.52s
        17           1.1571           0.0076            6.52s
        18           1.1502           0.0062            6.44s
        

## The deviance loss
Deviance loss is a commonly used loss function in binary classification problems. With a glance at its formula, we can easily unserstand why:

$$
L(y, p) = \left(y \log(p) + (1 - y) \log(1 - p)\right)
$$

where y is true class and p is statistical probability.




## F-1 score
F-1 score uses precision(ratio of true possitives to true possitves and false possitives) and recall(ratio of true possitives to true possitves and false negatives) scores to prvoide a balance between them:

$$F1 = 2 \times \frac{Precision \times Recall}{Precision + Recall}$$


In [9]:
# scores = cross_val_score(reg, x_train, y_train, cv=5, scoring='f1_macro')
# print("cross validation scores(F-1) where k=5: ", scores)

In [10]:
# scores = cross_val_score(reg, x_train, y_train, cv=10, scoring='f1_macro')
# print("cross validation scores(F-1) where k=10: ", scores)

## Initial evaluataion result
As demonstrated above, whether employing Gradient Boosting with or without cross-validation, the F1 score hovers around 0.75. While this performance is acceptable, there is room for improvement.

# Second model: Logistic regression
While Logistic Regression is typically considered a more linear model compared to ensemble methods, it remains a highly prevalent choice in classification problems. It offers several distinct advantages, such as strong interpretability, feature importance insights, and the ability to not only make binary classifications but also provide class probabilities. This probabilistic aspect can prove particularly valuable in certain situations."







In [11]:
def logisticRegressionModel(dataset):
  x_train, x_test, y_train, y_test = get_data(dataset)
  log_reg = LogisticRegression(random_state=32, solver='sag', multi_class='multinomial', verbose=2, max_iter=500)
  log_reg.fit(x_train, y_train)
  y_pred_log_reg = log_reg.predict(x_test)
  report_log_reg = classification_report(y_test, y_pred_log_reg)
  print(log_reg.coef_)
  print(report_log_reg)

In [12]:
logisticRegressionModel(raw_dataset)

convergence after 182 epochs took 5 seconds
[[ 0.3682435   0.29301023  0.68196175  0.03713967 -0.00414803  0.08711372
   0.11512442 -0.01482644 -0.0180894  -0.04394051 -0.36520999  0.03005322
   0.00677025  0.29309753 -0.00264087 -0.00402706  0.06804295  0.13130041
   0.07615977 -0.01306131 -0.02943937]]
              precision    recall  f1-score   support

         0.0       0.75      0.73      0.74      7010
         1.0       0.74      0.76      0.75      7129

    accuracy                           0.74     14139
   macro avg       0.74      0.74      0.74     14139
weighted avg       0.74      0.74      0.74     14139



## Evaluation result
Logistic regression exhibited slightly lower performance compared to Gradient Boosting, indicating that additional data preprocessing may be necessary to enhance model outcomes.

### Checking for class imbalance

In [13]:
np.bincount(y)

array([35346, 35346])

## Standardizing features
In this section we standardize featuers that their domain may mislead oue models.

In [14]:
columns_to_standardize = list(x.keys())

In [15]:
scaler = StandardScaler()
standarized_features = scaler.fit_transform(raw_dataset[columns_to_standardize])
standardized_dataset = pd.DataFrame()
standardized_dataset["Diabetes_binary"] = raw_dataset["Diabetes_binary"]
standardized_dataset[columns_to_standardize] = standarized_features

In [16]:
gradienBoostClassifierModel(standardized_dataset)

      Iter       Train Loss      OOB Improve   Remaining Time 
         1           1.3619           0.0245            7.32s


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py:280: FutureWarning: The loss parameter name 'deviance' was deprecated in v1.1 and will be removed in version 1.3. Use the new parameter name 'log_loss' which is equivalent.
  warnings.warn(


         2           1.3400           0.0224            7.08s
         3           1.3200           0.0203            7.00s
         4           1.3008           0.0178            6.98s
         5           1.2841           0.0163            6.97s
         6           1.2685           0.0151            6.96s
         7           1.2545           0.0136            6.89s
         8           1.2417           0.0127            6.82s
         9           1.2303           0.0117            6.77s
        10           1.2191           0.0112            6.73s
        11           1.2090           0.0101            6.69s
        12           1.1982           0.0100            6.62s
        13           1.1890           0.0094            6.55s
        14           1.1793           0.0088            6.49s
        15           1.1707           0.0078            6.43s
        16           1.1641           0.0067            6.39s
        17           1.1571           0.0076            6.35s
        

In [17]:
logisticRegressionModel(standardized_dataset)

convergence after 35 epochs took 2 seconds
[[ 0.1826531   0.14632449  0.10666609  0.26429747 -0.0020377   0.02104763
   0.04085052 -0.00675311 -0.00880712 -0.01792627 -0.07395524  0.00629132
   0.00203814  0.32646677 -0.02152548 -0.04053807  0.02956818  0.06543934
   0.21726925 -0.01336101 -0.0640264 ]]
              precision    recall  f1-score   support

         0.0       0.75      0.73      0.74      7010
         1.0       0.74      0.76      0.75      7129

    accuracy                           0.74     14139
   macro avg       0.74      0.74      0.74     14139
weighted avg       0.74      0.74      0.74     14139



## Normalizing features
Standardization helped the convergance of our model, but didn't countribute to the evaluation metrics. Now we try with normalized data.

In [18]:
columns_to_normalize = list(x.keys())

In [19]:
min_max_scaler = MinMaxScaler()
normalized_features = min_max_scaler.fit_transform(raw_dataset[columns_to_standardize])
normalized_dataset = pd.DataFrame()
normalized_dataset["Diabetes_binary"] = raw_dataset["Diabetes_binary"]
normalized_dataset[columns_to_standardize] = normalized_features

In [20]:
gradienBoostClassifierModel(normalized_dataset)

      Iter       Train Loss      OOB Improve   Remaining Time 

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py:280: FutureWarning: The loss parameter name 'deviance' was deprecated in v1.1 and will be removed in version 1.3. Use the new parameter name 'log_loss' which is equivalent.
  warnings.warn(



         1           1.3619           0.0245            9.55s
         2           1.3400           0.0224            9.48s
         3           1.3200           0.0203            9.33s
         4           1.3008           0.0178            9.23s
         5           1.2841           0.0163            9.08s
         6           1.2685           0.0151            9.04s
         7           1.2545           0.0136            8.95s
         8           1.2417           0.0127            8.99s
         9           1.2303           0.0117            8.87s
        10           1.2191           0.0112            8.86s
        11           1.2090           0.0101            8.78s
        12           1.1982           0.0100            8.75s
        13           1.1890           0.0094            8.67s
        14           1.1793           0.0088            8.67s
        15           1.1707           0.0078            8.56s
        16           1.1641           0.0067            8.52s
       

In [21]:
logisticRegressionModel(normalized_dataset)

convergence after 38 epochs took 1 seconds
[[ 0.36918431  0.29307524  0.68381231  3.15904998 -0.00423009  0.08667325
   0.11512165 -0.01518912 -0.01825656 -0.04387366 -0.36555518  0.03063572
   0.00687011  1.17187081 -0.07924642 -0.1206813   0.06894083  0.13120518
   0.91025488 -0.06528242 -0.2056759 ]]
              precision    recall  f1-score   support

         0.0       0.75      0.73      0.74      7010
         1.0       0.74      0.76      0.75      7129

    accuracy                           0.74     14139
   macro avg       0.74      0.74      0.74     14139
weighted avg       0.74      0.74      0.74     14139



# What happened?
It turned out that algorithms like Logitstic regression and Gradientboost are robust to data scale due to a number of factors like their loss functions, use of decision trees and regularization factors, etc. So we have to find another way to reach our goal.

# Next model: DNN
neural networks are the master of finding complex relations between featurse. In addition to that, they can be combined with various functionalities to improve model's behavoir even further, e.g. optimizers, regularization factors, etc.

In [86]:
def dnnModel(dataset):
  x_train, x_test, y_train, y_test = get_data(dataset)
  model = Sequential()
  model.add(Dense(64, input_dim=x_train.shape[1], activation=LeakyReLU(alpha=0.1), kernel_initializer=he_normal()))
  model.add(Dropout(0.5))
  model.add(Dense(128, activation='relu'))
  model.add(Dense(32, activation='relu', kernel_regularizer=l1(0.1)))
  model.add(Dense(1, activation='sigmoid'))
  adam = Adagrad(learning_rate=0.1)
  model.compile(loss='binary_crossentropy', optimizer=adam, metrics=[Precision(), Recall()])
  model.fit(x_train, y_train, epochs=100, verbose=2, validation_split=0.1, batch_size=100,)
  res = model.evaluate(x_test, y_test)
  print("binary cross-entropy loss : ", res[0], " precision: ", res[1], " recal: ", res[2])

In [87]:
dnnModel(standardized_dataset)

Epoch 1/100
509/509 - 1s - loss: 2.3903 - precision_3: 0.6628 - recall_3: 0.6294 - val_loss: 1.4111 - val_precision_3: 0.7265 - val_recall_3: 0.7981 - 1s/epoch - 3ms/step
Epoch 2/100
509/509 - 1s - loss: 1.2994 - precision_3: 0.7163 - recall_3: 0.7952 - val_loss: 1.1808 - val_precision_3: 0.7140 - val_recall_3: 0.8249 - 650ms/epoch - 1ms/step
Epoch 3/100
509/509 - 1s - loss: 1.1139 - precision_3: 0.7187 - recall_3: 0.7987 - val_loss: 1.0265 - val_precision_3: 0.7211 - val_recall_3: 0.8006 - 683ms/epoch - 1ms/step
Epoch 4/100
509/509 - 1s - loss: 1.0158 - precision_3: 0.7204 - recall_3: 0.8041 - val_loss: 0.9819 - val_precision_3: 0.7178 - val_recall_3: 0.8142 - 651ms/epoch - 1ms/step
Epoch 5/100
509/509 - 1s - loss: 0.9505 - precision_3: 0.7198 - recall_3: 0.8015 - val_loss: 0.8984 - val_precision_3: 0.7191 - val_recall_3: 0.8092 - 668ms/epoch - 1ms/step
Epoch 6/100
509/509 - 1s - loss: 0.9063 - precision_3: 0.7223 - recall_3: 0.8061 - val_loss: 0.8985 - val_precision_3: 0.7012 - val_r

## Result
As we saw, different models are not showing significant result improvements. So we may need to make some changes to our data

## The correlation matrix and its usage
Correlation matrix simply explains the relationship between columns of a dataset. The correlation coefficient ranges between -1 and 1. A correlation coefficient of 1 indicates a perfect positive correlation, meaning that the two variables increase or decrease together in a linear relationship. A correlation coefficient of -1 indicates a perfect negative correlation, meaning that the two variables move in opposite directions in a linear relationship. A correlation coefficient close to 0 suggests no linear relationship between the variables.

This matrix can be helpful when finding an optimal subset of features.

In [24]:
def sort_correlations(correlation_matrix):
  return sorted(correlation_matrix.items(), key=lambda x:abs(x[1]))

In [25]:
def get_correlations(dataset):
  columns = dataset.keys()
  correlation = dataset[columns].corr()
  return correlation["Diabetes_binary"]

In [31]:
correlation_map = sort_correlations(get_correlations(raw_dataset))

In [47]:
correlation_map

[('AnyHealthcare', 0.02319074853112824),
 ('NoDocbcCost', 0.040976573266643494),
 ('Sex', 0.044412858371260695),
 ('Fruits', -0.05407655628666651),
 ('Veggies', -0.07929314561269872),
 ('Smoker', 0.08599896420800192),
 ('MentHlth', 0.08702877147509416),
 ('HvyAlcoholConsump', -0.09485313995926549),
 ('CholCheck', 0.11538161710270915),
 ('Stroke', 0.12542678468516733),
 ('PhysActivity', -0.15866560486405157),
 ('Education', -0.17048063498806143),
 ('HeartDiseaseorAttack', 0.21152340436022687),
 ('PhysHlth', 0.21308101903810317),
 ('Income', -0.2244487149638171),
 ('DiffWalk', 0.272646006159808),
 ('Age', 0.27873806628188813),
 ('HighChol', 0.28921280708865016),
 ('BMI', 0.29337274476103575),
 ('HighBP', 0.3815155489073117),
 ('GenHlth', 0.4076115984949182),
 ('Diabetes_binary', 1.0)]

In [69]:
keep_features = map(lambda b: b[0], filter(lambda a: abs(a[1]) > 0.25, correlation_map))

In [70]:
modified_dataset = raw_dataset[list(keep_features)]

In [71]:
logisticRegressionModel(modified_dataset)

convergence after 38 epochs took 0 seconds
[[0.0702186  0.08500237 0.29923042 0.03782045 0.39012718 0.30067847]]
              precision    recall  f1-score   support

         0.0       0.75      0.73      0.74      7010
         1.0       0.74      0.76      0.75      7129

    accuracy                           0.74     14139
   macro avg       0.74      0.74      0.74     14139
weighted avg       0.74      0.74      0.74     14139



# Feature reduction result: Logistic regression
By reducing the feature count using the correlation matrix and only keeping faetures that have more meaningful relationship with the target featurse, Logistic regression not only converged faster, it also kept its accuracy.

In [82]:
gradienBoostClassifierModel(modified_dataset)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py:280: FutureWarning: The loss parameter name 'deviance' was deprecated in v1.1 and will be removed in version 1.3. Use the new parameter name 'log_loss' which is equivalent.
  warnings.warn(


      Iter       Train Loss      OOB Improve   Remaining Time 
         1           1.3619           0.0245            3.49s
         2           1.3400           0.0224            3.49s
         3           1.3200           0.0203            3.49s
         4           1.3008           0.0178            3.51s
         5           1.2841           0.0163            3.56s
         6           1.2685           0.0151            3.56s
         7           1.2545           0.0136            3.50s
         8           1.2417           0.0127            3.46s
         9           1.2303           0.0117            3.52s
        10           1.2191           0.0112            3.48s
        11           1.2090           0.0101            3.45s
        12           1.1982           0.0100            3.43s
        13           1.1890           0.0094            3.46s
        14           1.1793           0.0088            3.46s
        15           1.1707           0.0078            3.42s
       

# Feature reduction result: GradientBoost
GradientBoost was also capable of keeping its performance after feature reduction. It is worthy of noting that tuning hyperparameters had a mild effect on this model but it was negligible.

In [88]:
dnnModel(modified_dataset)

Epoch 1/100
509/509 - 3s - loss: 3.1584 - precision_4: 0.5469 - recall_4: 0.5559 - val_loss: 1.4402 - val_precision_4: 0.6672 - val_recall_4: 0.8528 - 3s/epoch - 5ms/step
Epoch 2/100
509/509 - 1s - loss: 1.2957 - precision_4: 0.6091 - recall_4: 0.6824 - val_loss: 1.1607 - val_precision_4: 0.6665 - val_recall_4: 0.8829 - 666ms/epoch - 1ms/step
Epoch 3/100
509/509 - 1s - loss: 1.0902 - precision_4: 0.6516 - recall_4: 0.6901 - val_loss: 0.9929 - val_precision_4: 0.7289 - val_recall_4: 0.7537 - 665ms/epoch - 1ms/step
Epoch 4/100
509/509 - 1s - loss: 0.9712 - precision_4: 0.6779 - recall_4: 0.6978 - val_loss: 0.8886 - val_precision_4: 0.7095 - val_recall_4: 0.8149 - 658ms/epoch - 1ms/step
Epoch 5/100
509/509 - 1s - loss: 0.9002 - precision_4: 0.6904 - recall_4: 0.7011 - val_loss: 0.8506 - val_precision_4: 0.7366 - val_recall_4: 0.7501 - 659ms/epoch - 1ms/step
Epoch 6/100
509/509 - 1s - loss: 0.8531 - precision_4: 0.6990 - recall_4: 0.7087 - val_loss: 0.7980 - val_precision_4: 0.7094 - val_r

# Feature reduction result: DNN
DNN also proved to be consistant after feature reduction. It even had a mild improvement(which is again, negligible).

# Feature reduction overall result
So to conclude, we were able to predict our target feature with an acceptable accuracy even after feature reduction. The following is the list of remaining features which proved to be decisive: DiffWalk, Age, HighChol, BMI, HighBP, GenHlth


In [ ]:
# to do: search for improvement techniques
# rename functions to snake_case
# save final cooef